In [1]:
import os
import sqlite3
import hashlib
import google.generativeai as google_genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

from typing import List, Optional

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# google_genai.configure(api_key=os.getenv('GOOGLE_API_KEY'), transport='rest')
google_genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

In [3]:
for m in google_genai.list_models():
    print(m.name)

RetryError: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B2404:6800:4009:820::200a%5D:443: Network is unreachable

In [4]:
llm = google_genai.GenerativeModel('gemini-1.0-pro')

In [5]:
llm.generate_content("The quick brown fox jumps over the lazy dog")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'The quick brown fox jumps over the lazy dog.'}], 'role': 'model'}, 'finish_reason': 1, 'index': 0, 'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 2, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'token_count': 0, 'grounding_attributions': []}], 'prompt_feedback': {'safety_ratings': [{'category': 9, 'probability': 1, 'blocked': False}, {'category': 8, 'probability': 1, 'blocked': False}, {'category': 7, 'probability': 1, 'blocked': False}, {'category': 10, 'probability': 1, 'blocked': False}], 'block_reason': 0}}),
)

In [6]:
# Setup Gemini Model
def setup_gemini_model(
    model_name:str,
    temperature:float,
    max_output_tokens:int,
    top_p:float,
    top_k:int,
    stop_sequence:Optional[List[str]]=None,
) -> google_genai.GenerativeModel:
    
    # Setup Safety Settings
    safety_settings = {
        # HarmCategory.HARM_CATEGORY_DANGEROUS: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_DEROGATORY: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_MEDICAL: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_SEXUAL: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_TOXICITY: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
        # HarmCategory.HARM_CATEGORY_VIOLENCE: HarmBlockThreshold.BLOCK_NONE,
    }
    
    # Setup Model Config
    generation_config = google_genai.GenerationConfig(
        candidate_count=1,
        stop_sequences=stop_sequence,
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
    )
    
    # Create a new model
    gemini_model = google_genai.GenerativeModel(
        model_name=model_name,
        safety_settings=safety_settings,
        generation_config=generation_config
        )

    return gemini_model

In [7]:
llm = setup_gemini_model(
    model_name='gemini-1.0-pro',
    temperature=0,
    max_output_tokens=1024,
    top_p=1,
    top_k=50,
)

In [8]:
print(llm)

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={'candidate_count': 1, 'max_output_tokens': 1024, 'temperature': 0, 'top_p': 1, 'top_k': 50},
    safety_settings={<HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 10>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HARASSMENT: 7>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_HATE_SPEECH: 8>: <HarmBlockThreshold.BLOCK_NONE: 4>, <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 9>: <HarmBlockThreshold.BLOCK_NONE: 4>},
    tools=None,
)


In [9]:
llm.count_tokens("The quick brown fox jumps over the lazy dog").total_tokens

9